In [86]:
import numpy as np
import pandas as pd
import os.path
from os import path
import csv
from pathlib import Path
import pickle
import os
from datetime import datetime
pd.set_option('display.max_rows', 500)

In [87]:
def composition_df_from_csv(file,composition_label):
    if path.exists(file):
        prices = pd.read_csv(csv_file, error_bad_lines=False, header=10)
        cols = prices.columns
        prices[cols] = prices[cols].replace({'\$': '',',': ''}, regex=True)
        prices.dropna(inplace=True)
        prices['composition_label'] = composition_label
        prices['composition_label'] = pd.to_datetime(prices['composition_label'])
        #prices.rename(columns={"Weights (%)": "Weights", "Asset  Class": "Asset_Class", "Market Share": "Market_Share", "Notional Value": "Notional_Value"}, errors="raise")
        return prices
    else:
        print(f"{file} doesn't exist")

In [88]:
all_tickers = []

combined_df = None
all_combined_df = None

base_dir = "../../data/rawdata/fund-composition-as-of/eoy/"
display_columns = ['Ticker','Sector','composition_label'] # 'Name','Asset Class','Weight (%)',

for file in os.listdir(base_dir):
    #print(f"evaluating {file}")
    if file.endswith('.csv'):
        print(f"\tprocessing {file}\n")
        date_str = file.replace('.csv','')
        composition_label = datetime.strptime(date_str, '%Y%m%d').strftime('%m/%d/%Y')
        csv_file = base_dir + file
        odf = composition_df_from_csv(csv_file,composition_label)
        all_tickers.extend(odf['Ticker'].tolist())
        #print(odf[display_columns].groupby(['Sector']).count()) # THis works!
        composition = odf[['Ticker','Sector','composition_label']]
        composition  = composition[composition['Sector'] == '-']
        combined_df = pd.concat([combined_df,composition], axis="rows", join="inner")
        all_combined_df = pd.concat([all_combined_df,odf], axis="rows", join="inner")

# mylist = list(dict.fromkeys(all_tickers))
# #print(mylist)
# #print(len(mylist))
        
   

	processing 20061229.csv

	processing 20071231.csv

	processing 20081231.csv

	processing 20091231.csv

	processing 20110228.csv

	processing 20111230.csv

	processing 20121231.csv

	processing 20131231.csv

	processing 20141231.csv

	processing 20151231.csv

	processing 20161230.csv

	processing 20171229.csv

	processing 20181231.csv



# Tickers that have a hyphen ( - ) character as Sector, by Composition
> Just throw out 2006 all together, or do a fillna based on the other years

In [89]:
combined_df.groupby(['composition_label']).count()
# combined_df.dtypes

,Ticker,Sector
composition_label,,
2006-12-29,2964,2964
2009-12-31,1,1
2011-02-28,1,1
2012-12-31,1,1
2013-12-31,31,31


# See all Tickers/Composition dates with "-" Sector
Excluding for 2006, which we know is junk

In [90]:
combined_df['composition_label'] = combined_df[combined_df['composition_label'] != 12/29/2006]

# All ETF Compositions
> 38700 Rows

In [91]:
my_cols = ['Ticker','Name','Asset Class','Weight (%)','Sector','Exchange','composition_label']
all_combined_df[my_cols]

,Ticker,Name,Asset Class,Weight (%),Sector,Exchange,composition_label
0,P5N994,Petrocorp Inc Escrow,Equity,0.00,-,Non-Nms Quotation Service (Nnqs),2006-12-29
1,TLCP,TELECORP PCS INC - ESCROW,Equity,0.00,-,NASDAQ,2006-12-29
2,REV&,REVLON INC RIGHT 01/19/2007,Equity,0.00,-,New York Stock Exchange Inc.,2006-12-29
3,TBV,TIENS BIOTECH GROUP (USA) INC.,Equity,0.00,-,Nyse Mkt Llc,2006-12-29
4,TTES,T-3 ENERGY SERVICES,Equity,0.00,-,NASDAQ,2006-12-29
...,...,...,...,...,...,...,...
3001,-,RESTRICTED CONTRA FERROGLOBE UNIT,Equity,0.00,Materials,NASDAQ,2018-12-31
3002,RTYH9,RUSSELL 2000 EMINI CME MAR 19,Futures,0.00,Cash and/or Derivatives,Chicago Mercantile Exchange,2018-12-31
3003,ESH9,S&P500 EMINI MAR 19,Futures,0.00,Cash and/or Derivatives,Index And Options Market,2018-12-31
3004,P5N994,Petrocorp Inc Escrow,Equity,0.00,Materials,Non-Nms Quotation Service (Nnqs),2018-12-31


# how many have weight's of 0?
> 18,890

In [92]:
a = all_combined_df
a[a['Weight (%)'] <= 0]

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange,composition_label
0,P5N994,Petrocorp Inc Escrow,Equity,0.00,0.00,1248.00,0.12,0.00,-,-,US71645N9947,Non-Nms Quotation Service (Nnqs),2006-12-29
1,TLCP,TELECORP PCS INC - ESCROW,Equity,0.00,0.00,6715.00,0.67,1.00,-,9926776,-,NASDAQ,2006-12-29
2,REV&,REVLON INC RIGHT 01/19/2007,Equity,0.00,0.05,27326.00,1366.30,1366.00,-,B1L9Y44,-,New York Stock Exchange Inc.,2006-12-29
3,TBV,TIENS BIOTECH GROUP (USA) INC.,Equity,0.00,3.93,633.00,2487.69,2488.00,-,2975968,US88650T1043,Nyse Mkt Llc,2006-12-29
4,TTES,T-3 ENERGY SERVICES,Equity,0.00,22.05,221.00,4873.05,4873.00,-,2828125,US87306E1073,NASDAQ,2006-12-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,-,RESTRICTED CONTRA FERROGLOBE UNIT,Equity,0.00,0.00,15827.00,0.16,0.16,Materials,-,-,NASDAQ,2018-12-31
3002,RTYH9,RUSSELL 2000 EMINI CME MAR 19,Futures,0.00,1349.00,31.00,0.00,2090950.00,Cash and/or Derivatives,-,-,Chicago Mercantile Exchange,2018-12-31
3003,ESH9,S&P500 EMINI MAR 19,Futures,0.00,2505.20,119.00,0.00,14905940.00,Cash and/or Derivatives,-,-,Index And Options Market,2018-12-31
3004,P5N994,Petrocorp Inc Escrow,Equity,0.00,0.00,1248.00,0.01,0.01,Materials,-,US71645N9947,Non-Nms Quotation Service (Nnqs),2018-12-31


# Weight of 0, grouped by composition

In [93]:
a[a['Weight (%)'] <= 0].filter(['Ticker','composition_label']).groupby('composition_label').count()

,Ticker
composition_label,
2006-12-29,1335
2007-12-31,1409
2008-12-31,1480
2009-12-31,1479
2011-02-28,1397
2011-12-30,1463
2012-12-31,1464
2013-12-31,1420
2014-12-31,1491


# Dispite half of the weights being 0 in each composition, the Weights add up to 98-97%
> Why not call it the Russell 1500 :)

In [94]:
a[['composition_label','Weight (%)']].groupby('composition_label').sum()

,Weight (%)
composition_label,
2006-12-29,97.94
2007-12-31,98.30
2008-12-31,97.54
2009-12-31,98.05
2011-02-28,97.93
2011-12-30,97.90
2012-12-31,98.07
2013-12-31,98.01
2014-12-31,98.26


# Drop empty weights

In [95]:
a.dropna(subset=['Weight (%)'], inplace=True)
a

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange,composition_label
0,P5N994,Petrocorp Inc Escrow,Equity,0.00,0.00,1248.00,0.12,0.00,-,-,US71645N9947,Non-Nms Quotation Service (Nnqs),2006-12-29
1,TLCP,TELECORP PCS INC - ESCROW,Equity,0.00,0.00,6715.00,0.67,1.00,-,9926776,-,NASDAQ,2006-12-29
2,REV&,REVLON INC RIGHT 01/19/2007,Equity,0.00,0.05,27326.00,1366.30,1366.00,-,B1L9Y44,-,New York Stock Exchange Inc.,2006-12-29
3,TBV,TIENS BIOTECH GROUP (USA) INC.,Equity,0.00,3.93,633.00,2487.69,2488.00,-,2975968,US88650T1043,Nyse Mkt Llc,2006-12-29
4,TTES,T-3 ENERGY SERVICES,Equity,0.00,22.05,221.00,4873.05,4873.00,-,2828125,US87306E1073,NASDAQ,2006-12-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,-,RESTRICTED CONTRA FERROGLOBE UNIT,Equity,0.00,0.00,15827.00,0.16,0.16,Materials,-,-,NASDAQ,2018-12-31
3002,RTYH9,RUSSELL 2000 EMINI CME MAR 19,Futures,0.00,1349.00,31.00,0.00,2090950.00,Cash and/or Derivatives,-,-,Chicago Mercantile Exchange,2018-12-31
3003,ESH9,S&P500 EMINI MAR 19,Futures,0.00,2505.20,119.00,0.00,14905940.00,Cash and/or Derivatives,-,-,Index And Options Market,2018-12-31
3004,P5N994,Petrocorp Inc Escrow,Equity,0.00,0.00,1248.00,0.01,0.01,Materials,-,US71645N9947,Non-Nms Quotation Service (Nnqs),2018-12-31


# Confirm that Weight (%) is a numeric type

In [96]:
a.dtypes

Ticker                       object
Name                         object
Asset Class                  object
Weight (%)                  float64
Price                        object
Shares                       object
Market Value                 object
Notional Value               object
Sector                       object
SEDOL                        object
ISIN                         object
Exchange                     object
composition_label    datetime64[ns]
dtype: object

In [85]:
a = a[a['Weight (%)'] <= 0]
a

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange,composition_label


# Dropping Weight (%) <= 0 resulted in 18890 rows (from 38700) 

In [49]:
a['Notional Value'] = pd.to_numeric(a['Notional Value'])
a = a[a['Notional Value'] <= 0]


ValueError: Unable to parse string "-" at position 1335